<center><img src="https://github.com/DACSS-Spatial/data_forSpatial/raw/main/logo.png" width="700"></center>

<a target="_blank" href="https://colab.research.google.com/github/DACSS-Spatial/GDF_OPS_applications/blob/main/gasandschools.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Moving Gas Stations away from Schools in Boston, MA

# Getting ready

## Installations needed

In [ ]:
# !pip install osmnx mapclassify

## Data needed

### Official data preprocessing

We need the boston map. As we are going to analize schools and gas stations we must omit the water bodies. A good option is:


*   https://www.mass.gov/info-details/massgis-data-municipalities

You will select the shapefile for Boston from there. The file returned will be a zip, which I have saved on GitHub from where I read it:



In [ ]:
import geopandas as gpd

filez="https://github.com/DACSS-Spatial/data_forSpatial/raw/refs/heads/main/BOSTON/GISDATA.TOWNSSURVEY_POLYM.zip"
boston=gpd.read_file(filez)
boston.info()

As you see, it is just one geometry:

In [ ]:
boston

The map we have has come projected:

In [ ]:
boston.crs

### Crowdsourced data pre processing

Now we need the schools and gas stations. Let's become familiar with OpenStreet Maps [API for Python](https://osmnx.readthedocs.io/en/stable/), which for our case would be very useful.

You can get the gas stations from Boston like this:


In [ ]:
# Get gas stations in Boston
import osmnx as ox

stations = ox.features_from_place("Boston, Massachusetts, USA",
                                tags={'amenity': 'fuel'})

Notice we are getting several gas stations, already as a GDF:

In [ ]:
stations.info()

OSM gave you stations unprojected:

In [ ]:
stations.crs

Notice the multi-index:

In [ ]:
stations

In the multi index you see **nodes**, **relations**, and **ways**:

In [ ]:
stations.index

Some formatting may be needed at this stage:

- This will put current indexes back into the data, and numbers will appear:

In [ ]:
stations.reset_index(drop=False,inplace=True)
stations.head()

- Keep some columns:

In [ ]:
colsToKeep=['element','name','geometry']
fuelBoston=stations[colsToKeep].copy()

- Reproject the data:

In [ ]:
Boston_crs = "EPSG:26986"

fuelBoston = fuelBoston.to_crs(Boston_crs)

- Review:

In [ ]:
fuelBoston.info()

- From above, columns **name** of the gas station has missing values. We should rename those missing values:

In [ ]:
fuelBoston.fillna({'name':'Unknown'},inplace=True)

- We usually use the names as row index. But row index should not have duplicates. Since several 'names' are the same (_Unknown_), we have to create new names:

In [ ]:
fuelBoston.index.astype(str)+"_"+fuelBoston.name

Then,

In [ ]:
fuelBoston['newname']=fuelBoston.index.astype(str)+"_"+fuelBoston.name
fuelBoston.set_index('newname',inplace=True)
fuelBoston.drop(columns=['name'],inplace=True)
fuelBoston.head()

See that **nodes** are points, **ways** are lines or simple shapes built from points, and **relations** are conceptual containers that group any of the other elements to represent complex features.

Let me compare with the geometries available:


In [ ]:
import pandas as pd

pd.crosstab(fuelBoston.element,fuelBoston.geometry.geom_type, margins=True)


This is the 'relation' we have:

In [ ]:
#original data
stations[stations.element=='relation']

In [ ]:
fuelBoston[fuelBoston.element=='relation'].plot()

Let me get the schools:

In [ ]:
# Get schools in Boston
schools = ox.features_from_place("Boston, Massachusetts, USA",
                                tags={'amenity': 'school'})
schools.info()

Let's reformat this as we did before:

In [ ]:
schools.reset_index(inplace=True)
schoolBoston=schools[['element','name','geometry']].copy()
schoolBoston.fillna({'name':'Unknown'},inplace=True)
schoolBoston['newname']=schoolBoston.index.astype(str)+"_"+schoolBoston.name
schoolBoston.set_index('newname',inplace=True)
schoolBoston.drop(columns=['name'],inplace=True)
schoolBoston = schoolBoston.to_crs(Boston_crs)
schoolBoston.head()


See element vs geo:

In [ ]:
pd.crosstab(schoolBoston.element,schoolBoston.geometry.geom_type, margins=True)

In [ ]:
#original data
schools[(schools.element=='relation') & (schools.geometry.geom_type=='MultiPolygon')]

In [ ]:
schoolBoston[(schoolBoston.element=='relation') & (schoolBoston.geometry.geom_type=='MultiPolygon')].plot()

In [ ]:
base=boston.explore(tiles='cartodbpositron',color='lightblue')
schoolBoston.explore(m=base,color='k')
fuelBoston.explore(m=base,color='red')

# Explore proximity:


Let's compute a couple of distance matrices.

- The distance among fuel stations

In [ ]:
D_Matrix_fuel_fuel=fuelBoston.geometry.apply\
(lambda station: fuelBoston.geometry.distance(station))

D_Matrix_fuel_fuel

From here, we can compute the minimal distance among those gas stations:

In [ ]:
D_Matrix_fuel_fuel.replace(0,None,inplace=True) # avoid the zero
D_Matrix_fuel_fuel.min(axis=1).sort_values().head(10)

- The distance among gas stations and schools:

In [ ]:
D_Matrix_fuel_school=fuelBoston.geometry.apply\
(lambda station: schoolBoston.geometry.distance(station))

D_Matrix_fuel_school

# Decision 1: Find the stations that should go away



Now, we could compute the minimal distance from a gas station to a school, and sort the stations by that value:

In [ ]:
D_Matrix_fuel_school.min(axis=1).sort_values().head(10)

By the previous exploration, we may decide that no station should at 100 or less from a school:

In [ ]:
mindDist=100

# Decision 2: Secure perimeter of every school

Let's create the safe area around the school. This requires **buffer**:

In [ ]:
schoolBoston_buffered=schoolBoston.buffer(mindDist)
schoolBoston_buffered

In [ ]:
#remember we have
type(schoolBoston_buffered)

Turning GS into GDF:

In [ ]:
secured_schoolBoston=gpd.GeoDataFrame(geometry=schoolBoston_buffered)
secured_schoolBoston

Buffers created polygons:

In [ ]:
secured_schoolBoston.geometry.geom_type.value_counts()

# Decision 3: Standardized Gas stations geometries

Not all gas stations are points:

In [ ]:
fuelBoston.geometry.geom_type.value_counts()

The point is a simple representation. Then let's make a buffer of 10 meters for every station to make sure the station is selected even if we just had one of its points.

In [ ]:
fuelBoston_allPoly=fuelBoston.copy()
fuelBoston_allPoly['geometry'] = [
    station.buffer(10) if station.geom_type == 'Point' else station
    for station in fuelBoston.geometry
]

In [ ]:
# rechecking
fuelBoston_allPoly.geometry.geom_type.value_counts()

# Decision 4: Overlay or SJoin to determine gas stations in trouble

This is great moment to test our understanding of these ops:

- do you want to move the points from the gas stations that are close to the schools?

In [ ]:
fuelBoston_allPoly.overlay(secured_schoolBoston,how='intersection',keep_geom_type=False)

- do you want to move the gas stations that are close to the schools?

In [ ]:
fuelBoston_allPoly.sjoin(secured_schoolBoston,how='inner',predicate='intersects')

The obvious choice is using **sjoin** + **intersects** (why not within?).

In [ ]:
gas_relocate=fuelBoston_allPoly.sjoin(secured_schoolBoston,how='inner',predicate='intersects')

It is possible the index may have duplicates:

In [ ]:
is_duplicate = gas_relocate.index.duplicated(keep=False)

gas_relocate[is_duplicate]

Remember our names are on the row index, then:

In [ ]:
# put the index as a column: reset_index(drop=False)
# drop duplicates in that column:drop_duplicates(subset='newname_left')
# column bask to index : set_index('newname_left')

gas_relocate.reset_index(drop=False).\
drop_duplicates(subset='newname_left').\
set_index('newname_left')

In [ ]:
#making the actual change
gas_relocate= gas_relocate.reset_index(drop=False).drop_duplicates(subset='newname_left').\
              set_index('newname_left')

Let's see both schools and those stations to relocate:

In [ ]:
base=secured_schoolBoston.sjoin(gas_relocate,how='inner',predicate='intersects').explore(color='yellow', tiles='cartodbpositron')
gas_relocate.explore(m=base,color='red')

# Decision 5: Find suitable places for the stations in trouble


We can not put the gas stations anywhere. They need to be situated on driveable routes, not inside a house or building.


- Find suitable locations along routes

Let's get the streets from Boston with the help of OSM:

In [ ]:
# Define the place
place = "Boston, Massachusetts, USA"

# Download the street network graph for Boston
G = ox.graph_from_place(place, network_type="drive")

# Convert the graph edges (streets) to a GeoDataFrame
streets = ox.graph_to_gdfs(G, nodes=False, edges=True)

streets.shape

You see we got a huge set of routes:

In [ ]:
streets.info()

Let's pay attention to the 'highway' column:

In [ ]:
streets.highway.value_counts().index

In [ ]:
GAS_STATION_ROAD_TYPES = [
    'motorway',      # High-speed limited access roads
    'primary',       # Major national roads
    'trunk',         # Important regional roads
    'motorway_link', # Motorway entrance/exit ramps
    'primary_link',  # Primary road connectors
    'trunk_link',     # Trunk road connectors
    ['motorway', 'trunk'],
    ['primary', 'motorway_link'],
    ['primary', 'primary_link']
]

suitable_roads = streets[streets.highway.isin(GAS_STATION_ROAD_TYPES)]

suitable_roads.shape

Let's reproject:

In [ ]:
suitable_roads = suitable_roads.to_crs(boston.crs)

The possible locations should be:
- Far enough from schools. They should be far from the GDF **secured_schoolBoston** (already buffered with a secure radius) .
- Far from other Gast stations. From **fuelBoston_allPoly** we could add a secured distance. This GDF was was buffered, but just to include the building, so we need to re buffer.

Then, we need to create a buffer around the stations:

In [ ]:
competitionDistance=100 # a safe value from above 'D_Matrix_fuel_fuel'
fuelBoston_allPoly_buffered=fuelBoston_allPoly.buffer(competitionDistance)

Do we have a GDF?

In [ ]:
type(fuelBoston_allPoly_buffered)

In [ ]:
secured_fuelBoston_allPoly=gpd.GeoDataFrame(geometry=fuelBoston_allPoly_buffered)


Let's combine the secured areas:



In [ ]:
secured_areas_dissolved=secured_fuelBoston_allPoly.overlay(secured_schoolBoston,how='union',keep_geom_type=False).dissolve()
secured_areas=gpd.GeoDataFrame(geometry=secured_areas_dissolved.geometry)

- Filter the roads. We can NOT choose a location that intersects with the secured areas:

Choose: **overlay** vs **sjoin** vs **clip**?

In [ ]:
suitable_roads.clip(secured_areas,keep_geom_type=False).plot()

In [ ]:
suitable_roads.overlay(secured_areas,keep_geom_type=False, how='difference').plot()

In [ ]:
suitable_roads.sjoin(secured_areas, predicate='intersects').plot()

Why **clip** is a poor choice?
- Keeps only the parts that overlap with the mask. We want the opposite - to remove the overlapping parts (this would give you only roads inside school zones (the exact opposite!))

Why **sjoin** is a poor choice?
- You do get the precise locations that should not be included, you get the whole row element (more area). Since sjoin does not have a **not-intersects** predicate, this will also be wrong:

```
bad_geoms = suitable_roads.clip(secured_areas)
good_segments = suitable_roads.overlay(bad_geoms, how='difference')
```

Then:

In [ ]:
good_routes_forGas_dissolved=suitable_roads.overlay(secured_areas,keep_geom_type=False, how='difference').dissolve()
good_routes_forGas=gpd.GeoDataFrame(geometry=good_routes_forGas_dissolved.geometry)
good_routes_forGas

In [ ]:
good_routes_forGas.plot()

We may choose a location from those places:

In [ ]:
# newCOMPETITORS=safe_fuelCompetitors_gdf.copy()
potential_Locations=good_routes_forGas.copy()
newPOLYGONS=[]
pointsComputed=1
attempts=1
while pointsComputed<=len(gas_relocate):
  candidatePoint=potential_Locations.sample_points(1)
  candidateStation=gpd.GeoDataFrame(geometry=candidatePoint.buffer(100))
  if candidateStation.overlay(secured_areas,how='intersection',keep_geom_type=False).empty:
    newPOLYGONS.append(candidateStation)
    pointsComputed+=1
    potential_Locations=potential_Locations.overlay(candidateStation, how='difference', keep_geom_type=False)
  if attempts>len(gas_relocate)*10:
    break
  attempts+=1

# some info
attempts,len(newPOLYGONS)



In [ ]:
newPOLYGONS

In [ ]:
pd.concat(newPOLYGONS)

In [ ]:
GoodLocationsGas_buffered_gdf=pd.concat(newPOLYGONS)
GoodLocationsGas_gdf=gpd.GeoDataFrame(geometry=GoodLocationsGas_buffered_gdf.centroid)

In [ ]:
# original suitable roads
base = suitable_roads.plot(color='yellow', figsize=(10, 10),zorder=1)

# non-overlapping buffers
GoodLocationsGas_buffered_gdf.plot(ax=base, edgecolor='red', marker="+", facecolor='none',zorder=2)

# center points of the buffers
GoodLocationsGas_gdf.plot(ax=base, color='red', marker="+", markersize=50,zorder=3)

# all stations
fuelBoston.plot(ax=base,color='k',zorder=4)
schoolBoston.plot(ax=base,color='green',zorder=5)

# stations to relocate
fuelBoston.clip(gas_relocate).plot(ax=base,color='magenta',zorder=5)


## Reverse geocoding

Just get some addresses:

In [ ]:
from geopy.geocoders import Nominatim
import time

# Initialize geocoder
geolocator = Nominatim(user_agent="boston_gas_stations")

def get_address(point):
  # Now coordinates are in degrees (lon, lat)
  lon = point.x
  lat = point.y
  location = geolocator.reverse((lat, lon), exactly_one=True, timeout=10)
  time.sleep(1)  # Rate limiting

  if location:
    return location.address
  else:
    return None # not found

In [ ]:
# Get addresses
GoodLocationsGas_4326 = GoodLocationsGas_gdf.to_crs('EPSG:4326')
GoodLocationsGas_gdf['address'] = GoodLocationsGas_4326.geometry.apply(get_address)

In [ ]:
GoodLocationsGas_gdf

______

[BACK TO MAIN MENU](https://dacss-spatial.github.io/GDF_OPS_applications/)